# Import Library

In [ ]:
import cv2
import numpy as np
import os
from matplotlib import pyplot as pyplot

# Directory Path

In [ ]:
target_path = './Images/Object.png'
data_folder = './Images/Data/'

# Preprocessing

In [ ]:
def preprocess_v1(img):
    gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    eq = cv2.equalizeHist(gray)
    blur = cv2.medianBlur(eq, 5)
    return blur

def preprocess_v2(img):
    gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    eq = cv2.equalizeHist(gray)
    blur = cv2.GaussianBlur(eq, (3, 3), 0)
    return blur

# Find Best Match

In [ ]:
def find_best_match(preprocess_func, title):

    target = cv2.imread(target_path)
    target = cv2.cvtColor(target, cv2.COLOR_BGR2GRAY)

    target_gray = preprocess_func(target)

    


